In [1]:
import numpy as np
import torch

## Introduction to Linear Regression
In this tutorial, we'll discuss one of the foundational algorithms in machine learning: Linear regression. We'll create a model that predicts crop yields for apples and oranges (target variables) by looking at the average temperature, rainfall, and humidity (input variables or features) in a region. Here's the training data:

linear-regression-training-data

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
Visually, it means that the yield of apples is a linear or planar function of temperature, rainfall and humidity:

linear-regression-graph

The learning part of linear regression is to figure out a set of weights w11, w12,... w23, b1 & b2 using the training data, to make accurate predictions for new data. The learned weights will be used to predict the yields for apples and oranges in a new region using the average temperature, rainfall, and humidity for that region.

## Training data

We can represent the training data using two matrices: inputs and targets, each with one row per observation, and one column per variable.

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
#chanching Fahrenheit to Celsius
inputs[:, 0] = (inputs[:,0]-32.) * 5/9

In [4]:
inputs

array([[ 22.777779,  67.      ,  43.      ],
       [ 32.77778 ,  88.      ,  64.      ],
       [ 30.555555, 134.      ,  58.      ],
       [ 38.88889 ,  43.      ,  37.      ],
       [ 20.555555,  96.      ,  70.      ]], dtype=float32)

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 22.7778,  67.0000,  43.0000],
        [ 32.7778,  88.0000,  64.0000],
        [ 30.5556, 134.0000,  58.0000],
        [ 38.8889,  43.0000,  37.0000],
        [ 20.5556,  96.0000,  70.0000]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Linear regression model from scratch

The weights and biases (w11, w12,... w23, b1 & b2) can also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable, i.e., yield of apples, and similarly, the second for oranges.

In [7]:
w = torch.randn((2,3), requires_grad = True)
b = torch.randn((2), requires_grad = True)
print(w)
print(b)

tensor([[ 1.2934,  1.0001,  1.2473],
        [-1.3848,  1.0593,  0.0908]], requires_grad=True)
tensor([-0.6508, -0.4331], requires_grad=True)


Model is a Matrixmultiplication:

y = x * w.T + b

In [8]:
def model(x):
    return x @ w.t() + b

@ represents matrix multiplication in PyTorch, and the .t() method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [9]:
print(inputs.shape)
print(w.shape)

torch.Size([5, 3])
torch.Size([2, 3])


In [10]:
#predictions
y = model(inputs)
print(y)

tensor([[149.4487,  42.9024],
        [209.5773,  53.2065],
        [245.2233, 104.4684],
        [138.8015,  -5.3769],
        [209.2531,  79.1512]], grad_fn=<AddBackward0>)


In [11]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


You can see a big difference between our model's predictions and the actual targets because we've initialized our model with random weights and biases. Obviously, we can't expect a randomly initialized model to just work.

## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (preds and targets).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the mean squared error (MSE).

In [12]:
# MSE

def mse(t1, t2):
    diff = t1-t2
    return torch.sum(diff * diff)/diff.numel()

In [13]:
loss = mse(y, targets)
print(loss)

tensor(7334.5649, grad_fn=<DivBackward0>)


Here’s how we can interpret the result: On average, each element in the prediction differs from the actual target by the square root of the loss. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the loss because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have requires_grad set to True. We'll see how this is useful in just a moment.

In [14]:
# Compute gradients
loss.backward()

The gradients are stored in the .grad property of the respective tensors. Note that the derivative of the loss w.r.t. the weights matrix is itself a matrix with the same dimensions.

In [15]:
# Gradients for weights
print(w)
print(w.grad)
print(b.grad)

tensor([[ 1.2934,  1.0001,  1.2473],
        [-1.3848,  1.0593,  0.0908]], requires_grad=True)
tensor([[ 3385.2444,  9942.5059,  6265.5107],
        [-1104.5380, -3098.4578, -2047.2346]])
tensor([114.2607, -37.1297])


## Adjust weights and biases to reduce the loss

The loss is a quadratic function of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's slope w.r.t. the weights and biases.

If a gradient element is positive:

* increasing the weight element's value slightly will increase the loss
* decreasing the weight element's value slightly will decrease the loss

If a gradient element is negative:

* increasing the weight element's value slightly will decrease the loss
* decreasing the weight element's value slightly will increase the loss

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of the gradient descent optimization algorithm that we'll use to improve our model (by descending along the gradient).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

In [16]:
eta = 1e-5
with torch.no_grad():
    w -= w.grad * eta
    b -= b.grad * eta

We multiply the gradients with a very small number (10^-5 in this case) to ensure that we don't modify the weights by a very large amount. We want to take a small step in the downhill direction of the gradient, not a giant leap. This number is called the learning rate of the algorithm.

We use torch.no_grad to indicate to PyTorch that we shouldn't track, calculate, or modify gradients while updating the weights and biases.

In [17]:
   # Let's verify that the loss is actually lower
loss = mse(y, targets)
print(loss) 

tensor(7334.5649, grad_fn=<DivBackward0>)


Before we proceed, we reset the gradients to zero by invoking the .zero_() method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke .backward on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [18]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can train the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [19]:
#1. Generate Predictions
preds = model(inputs)
print(preds)

tensor([[139.3208,  46.1106],
        [195.7072,  57.6058],
        [227.2308, 110.1456],
        [130.8903,  -2.8572],
        [194.6254,  83.7862]], grad_fn=<AddBackward0>)


In [20]:
# 2. Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(5787.0991, grad_fn=<DivBackward0>)


In [21]:
# 3. Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[ 3016.5591,  8731.5938,  5528.8315],
        [ -987.7362, -2715.2300, -1813.9417]])
tensor([101.3549, -33.0418])


In [22]:
# 4. Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [23]:
#new weights and biases
print(w)
print(b)

tensor([[ 1.2294,  0.8133,  1.1293],
        [-1.3639,  1.1175,  0.1294]], requires_grad=True)
tensor([-0.6529, -0.4324], requires_grad=True)


In [24]:
# 5. Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(4587.8267, grad_fn=<DivBackward0>)


We have already achieved a significant reduction in the loss merely by adjusting the weights and biases slightly using gradient descent.

## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. Let's train the model for 100 epochs.

In [25]:
# Train for 10000 epochs
for i in range(5000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

Once again, let's verify that the loss is now lower:

In [26]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(2.5992, grad_fn=<DivBackward0>)


The loss is now much lower than its initial value. Let's look at the model's predictions and compare them with the targets.

In [27]:
preds

tensor([[ 59.8841,  72.3891],
        [ 80.4678,  99.3838],
        [118.5030, 132.6960],
        [ 21.3332,  37.1186],
        [101.7879, 119.1882]], grad_fn=<AddBackward0>)

In [28]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The predictions are now quite close to the target variables. We can get even better results by training for a few more epochs.

In [29]:
jovian.commit(project='02-linear-regression')

<IPython.core.display.Javascript object>

[jovian] Updating notebook "marc1971/02-linear-regression" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/marc1971/02-linear-regression


'https://jovian.ai/marc1971/02-linear-regression'

# Linear regression using PyTorch built-ins

We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.

In [30]:
import torch.nn as nn

In [31]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [32]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

We are using 15 training examples to illustrate how to work with large datasets in small batches.

## Dataset and DataLoader

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [33]:
from torch.utils.data import TensorDataset

In [34]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

In [35]:
from torch.utils.data import DataLoader

In [36]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

We can use the data loader in a for loop. Let's look at an example.

In [37]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  88.,  64.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.]])
tensor([[ 81., 101.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 132.],
        [ 21.,  38.]])


In each iteration, the data loader returns one batch of data with the given batch size. If shuffle is set to True, it shuffles the training data before creating batches. Shuffling helps randomize the input to the optimization algorithm, leading to a faster reduction in the loss.

## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the nn.Linear class from PyTorch, which does it automatically.

In [38]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.4195,  0.3642, -0.5727],
        [-0.2867,  0.2560, -0.3702]], requires_grad=True)
Parameter containing:
tensor([ 0.3800, -0.2605], requires_grad=True)


PyTorch models also have a helpful .parameters method, which returns a list containing all the weights and bias matrices present in the model. For our linear regression model, we have one weight matrix and one bias matrix.

In [39]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.4195,  0.3642, -0.5727],
         [-0.2867,  0.2560, -0.3702]], requires_grad=True),
 Parameter containing:
 tensor([ 0.3800, -0.2605], requires_grad=True)]

We can use the model to generate predictions in the same way as before.

In [40]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 30.7780, -19.9582],
        [ 33.9502, -27.5176],
        [ 52.4624, -12.3754],
        [ 37.6381, -32.1942],
        [ 24.1991, -21.3839],
        [ 30.8333, -20.5009],
        [ 33.0133, -28.1438],
        [ 52.3091, -13.0323],
        [ 37.5828, -31.6516],
        [ 23.2069, -21.4674],
        [ 29.8411, -20.5844],
        [ 34.0055, -28.0603],
        [ 53.3993, -11.7492],
        [ 38.6303, -32.1107],
        [ 24.1438, -20.8412]], grad_fn=<AddmmBackward0>)

## Loss Function
Instead of defining a loss function manually, we can use the built-in loss function mse_loss.

In [41]:
# Import nn.functional
import torch.nn.functional as F

The nn.functional package contains many useful loss functions and several other utilities.

In [42]:
# Define loss function
loss_fn = F.mse_loss

Let's compute the loss for the current predictions of our model.

In [43]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8410.2432, grad_fn=<MseLossBackward0>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer optim.SGD. SGD is short for "stochastic gradient descent". The term stochastic indicates that samples are selected in random batches instead of as a single group.

In [44]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

Note that model.parameters() is passed as an argument to optim.SGD so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function fit that trains the model for a given number of epochs.

In [45]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

We use the data loader defined earlier to get batches of data for every iteration.

Instead of updating parameters (weights and biases) manually, we use opt.step to perform the update and opt.zero_grad to reset the gradients to zero.

We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. loss.item returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [46]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 389.4570
Epoch [20/100], Loss: 280.2106
Epoch [30/100], Loss: 348.4700
Epoch [40/100], Loss: 101.8021
Epoch [50/100], Loss: 205.7414
Epoch [60/100], Loss: 87.1533
Epoch [70/100], Loss: 56.9371
Epoch [80/100], Loss: 132.6912
Epoch [90/100], Loss: 85.1529
Epoch [100/100], Loss: 100.3610


Let's generate predictions using our model and verify that they're close to our targets.

In [47]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 58.9678,  71.5051],
        [ 77.1803,  96.2528],
        [126.7693, 141.3984],
        [ 30.9262,  43.4372],
        [ 87.5386, 107.6322],
        [ 57.7996,  70.4145],
        [ 76.0312,  95.4918],
        [126.5644, 141.5587],
        [ 32.0944,  44.5279],
        [ 87.5579, 107.9618],
        [ 57.8188,  70.7441],
        [ 76.0120,  95.1622],
        [127.9183, 142.1594],
        [ 30.9069,  43.1077],
        [ 88.7069, 108.7228]], grad_fn=<AddmmBackward0>)

In [48]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [49]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[54.4262, 67.5568]], grad_fn=<AddmmBackward0>)

The predicted yield of apples is 54.4 tons per hectare, and that of oranges is 67.5 tons per hectare.

In [50]:
jovian.commit(project='02-linear-regression')

<IPython.core.display.Javascript object>

[jovian] Updating notebook "marc1971/02-linear-regression" on https://jovian.ai/
[jovian] Committed successfully! https://jovian.ai/marc1971/02-linear-regression


'https://jovian.ai/marc1971/02-linear-regression'